In [ ]:
#Importar las librerías necesarias
from sage.all import *
import hashlib
import random

In [ ]:

#  UTILIDADES DE COMPROMISO (FCom) – versión demostrativa


def commit(value):
    """
    Crea un compromiso usando SHA-256.
    value: valor entero o polinomio convertible a string.
    Retorna:
        - digest: cadena hash
        - opening: semilla + valor (para abrir después)
    """
    seed = random.randbytes(16)
    msg = seed + str(value).encode()
    digest = hashlib.sha256(msg).hexdigest()
    return digest, (seed, value)


def verify_commit(digest, opening):
    """
    Verifica que el compromiso corresponda al valor abierto.
    """
    seed, value = opening
    msg = seed + str(value).encode()
    return hashlib.sha256(msg).hexdigest() == digest

In [ ]:
###############################################################
#                    AuthShare Class
###############################################################

class AuthShare:
    """
    Authenticated share <x> with MACs using key alpha.
    This is a simplified SPDZ-style authenticated share.
    """

    def __init__(self, F, delta, shares, macs):
        self.F = F
        self.delta = F(delta)
        self.shares = [F(s) for s in shares]
        self.macs = [F(m) for m in macs]

    def value(self):
        """Reconstruct the secret value x."""
        return sum(self.shares)

    def mac_sum(self):
        """Reconstruct the sum of MAC shares."""
        return sum(self.macs)



#Fase de Entrada
Un partícipe quiere ingresar un valor x, lo que se hace es usar el valor generado en la fase de preprocesamiento (〈r〉, [r]) donde 〈r〉es el valor autenticado por los otros participantes y [r] es el valor reconstruido y protegido


In [ ]:
###############################################################
#                       FASE DE ENTRADA                       #
###############################################################

def input_phase(F, players, x_input, R_share):
    """
    Implements:
        ε = x - r
        return <x> = <r> + ε
    """
    r_val = R_share.value()           # open r
    eps = F(x_input - r_val)          # public epsilon

    # Compute new shares: x_i = r_i + eps
    new_shares = [F(s + eps) for s in R_share.shares]

    # MACs remain identical (MAC(r) becomes MAC(x))
    new_macs = R_share.macs

    return AuthShare(F, R_share.delta, new_shares, new_macs), eps